In [1]:
## color_gen function was taken from inneka.com programming tutorial ##
# Dependencies and Setup
import pandas as pd
import numpy as np
import bokeh.plotting as bp
from bokeh.palettes import Category10
from bokeh.models import ColumnDataSource, Whisker
import itertools

# Hide warning messages in notebook
import warnings
warnings.filterwarnings('ignore')

# This will be used to loop through color scheme
def color_gen():
    for c in itertools.cycle(Category10[10]):
        yield c

# File to Load (Remember to Change These)
mouse_drug_data_to_load = "data/mouse_drug_data.csv"
clinical_trial_data_to_load = "data/clinicaltrial_data.csv"

# Read the Mouse and Drug Data and the Clinical Trial Data

mouse_df = pd.read_csv(mouse_drug_data_to_load)
clinic_df = pd.read_csv(clinical_trial_data_to_load)
# Combine the data into a single dataset

df = clinic_df.set_index('Mouse ID').join(mouse_df.set_index('Mouse ID'))
# Display the data table for preview

df.head()

,Timepoint,Tumor Volume (mm3),Metastatic Sites,Drug
Mouse ID,,,,
a203,0,45.000000,0,Infubinol
a203,5,48.508468,0,Infubinol
a203,10,51.852437,1,Infubinol
a203,15,52.777870,1,Infubinol
a203,20,55.173336,1,Infubinol


## Tumor Response to Treatment

In [2]:
#ean Tumor Volume Data Grouped by Drug and Timepoint 
tum_vol_mean = df.groupby(['Drug', 'Timepoint']).mean()['Tumor Volume (mm3)']
# Convert to DataFrame
tumor_reponse_df = pd.DataFrame({"Tumor Volume (mm3)":tum_vol_mean})
# Preview DataFrame
tumor_reponse_df

Tumor Volume (mm3)
Drug      Timepoint                    
Capomulin 0                   45.000000
          5                   44.266086
          10                  43.084291
          15                  42.064317
          20                  40.716325
...                                 ...
Zoniferol 25                  55.432935
          30                  57.713531
          35                  60.089372
          40                  62.916692
          45                  65.960888

[100 rows x 1 columns]

In [3]:
# Store the Standard Error of Tumor Volumes Grouped by Drug and Timepoint
tum_vol_sem = df.groupby(['Drug', 'Timepoint']).sem()['Tumor Volume (mm3)']
# Convert to DataFrame
tumor_reponse_sem = pd.DataFrame({"Tumor Volume (mm3)":tum_vol_sem})
# Preview DataFrame
tumor_reponse_sem


Tumor Volume (mm3)
Drug      Timepoint                    
Capomulin 0                    0.000000
          5                    0.448593
          10                   0.702684
          15                   0.838617
          20                   0.909731
...                                 ...
Zoniferol 25                   0.602513
          30                   0.800043
          35                   0.881426
          40                   0.998515
          45                   1.003576

[100 rows x 1 columns]

In [4]:
# Minor Data Munging to Re-Format the Data Frames
tumor_reponse_df_unstacked = tumor_reponse_df.unstack(level = 'Drug')['Tumor Volume (mm3)']
tumor_reponse_sem_unstacked = tumor_reponse_sem.unstack(level = 'Drug')['Tumor Volume (mm3)']

# Preview that Reformatting worked
tumor_reponse_df_unstacked.head()


Drug,Capomulin,Ceftamin,Infubinol,Ketapril,Naftisol,Placebo,Propriva,Ramicane,Stelasyn,Zoniferol
Timepoint,,,,,,,,,,
0,45.000000,45.000000,45.000000,45.000000,45.000000,45.000000,45.000000,45.000000,45.000000,45.000000
5,44.266086,46.503051,47.062001,47.389175,46.796098,47.125589,47.248967,43.944859,47.527452,46.851818
10,43.084291,48.285125,49.403909,49.582269,48.694210,49.423329,49.101541,42.531957,49.463844,48.689881
15,42.064317,50.094055,51.296397,52.399974,50.933018,51.359742,51.067318,41.495061,51.529409,50.779059
20,40.716325,52.157049,53.197691,54.920935,53.644087,54.364417,53.346737,40.238325,54.067395,53.170334


In [5]:
# Preview that Reformatting worked
tumor_reponse_sem_unstacked.head()

Drug,Capomulin,Ceftamin,Infubinol,Ketapril,Naftisol,Placebo,Propriva,Ramicane,Stelasyn,Zoniferol
Timepoint,,,,,,,,,,
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.448593,0.164505,0.235102,0.264819,0.202385,0.218091,0.231708,0.482955,0.239862,0.188950
10,0.702684,0.236144,0.282346,0.357421,0.319415,0.402064,0.376195,0.720225,0.433678,0.263949
15,0.838617,0.332053,0.357705,0.580268,0.444378,0.614461,0.466109,0.770432,0.493261,0.370544
20,0.909731,0.359482,0.476210,0.726484,0.595260,0.839609,0.555181,0.786199,0.621889,0.533182


In [6]:
# Generate the Plot (with Error Bars)
color = color_gen()

y_list = df.Drug.unique()

#calculate the upper and lower for the error bars
drug_val_dict = {'Capomulin': {'upper':[],'lower': [], 'base':[]},
                 'Ceftamin' : {'upper':[],'lower': [], 'base':[]},
                 'Infubinol': {'upper':[],'lower': [], 'base':[]},
                 'Ketapril' : {'upper':[],'lower': [], 'base':[]},
                 'Naftisol' : {'upper':[],'lower': [], 'base':[]},
                 'Placebo'  : {'upper':[],'lower': [], 'base':[]},
                 'Propriva' : {'upper':[],'lower': [], 'base':[]},
                 'Ramicane' : {'upper':[],'lower': [], 'base':[]},
                 'Stelasyn' : {'upper':[],'lower': [], 'base':[]},
                 'Zoniferol': {'upper':[],'lower': [], 'base':[]}
                }

for key, val in drug_val_dict.items():
    drug_val_dict[key]['lower'].append(tumor_reponse_df_unstacked[key].values - tumor_reponse_sem_unstacked[key].values)
    drug_val_dict[key]['upper'].append(tumor_reponse_df_unstacked[key].values - tumor_reponse_sem_unstacked[key].values)
    drug_val_dict[key]['base'].append(tumor_reponse_df_unstacked[key].values)
    
#print(drug_val_dict['Capomulin']['base'])
source = bp.ColumnDataSource(tumor_reponse_df_unstacked)

fig = bp.figure(title = 'Time Response to Treatment', x_axis_label='Time (Days)',x_range= (0,50), y_axis_label='Tumor Volume (mm3)', y_range=(30,80) )

i = 0
for i, c in zip(range(len(y_list)), color):
    fig.line('Timepoint', y_list[i], line_width=2, source=source, color = c)
    source_error = ColumnDataSource(drug_val_dict[y_list[i]])
    fig.add_layout(Whisker(source=source_error, upper ='upper', lower='lower', base = 'base', line_width = 4, line_color = 'black'))

bp.output_file('Tumor_Response.html', title='Tumor Response to Treatment') #to show in another window.
bp.save(fig)

try:
    bp.reset_output()
    bp.output_notebook()
    bp.show(fig)
except:
    bp.output_notebook()
    bp.show(fig)


Loading BokehJS ...

Loading BokehJS ...

![Tumor Response to Treatment](../Images/treatment.png)

## Metastatic Response to Treatment

In [7]:
# Store the Mean Met. Site Data Grouped by Drug and Timepoint 
meta_mean = df.groupby(['Drug', 'Timepoint']).mean()['Metastatic Sites']
# Convert to DataFrame
meta_mean_df = pd.DataFrame({"Metastatic Sites":meta_mean})
# Preview DataFrame
meta_mean_df


Metastatic Sites
Drug      Timepoint                  
Capomulin 0                  0.000000
          5                  0.160000
          10                 0.320000
          15                 0.375000
          20                 0.652174
...                               ...
Zoniferol 25                 1.687500
          30                 1.933333
          35                 2.285714
          40                 2.785714
          45                 3.071429

[100 rows x 1 columns]

In [8]:
# Store the Standard Error associated with Met. Sites Grouped by Drug and Timepoint 
meta_sem = df.groupby(['Drug', 'Timepoint']).sem()['Metastatic Sites']
# Convert to DataFrame
meta_sem_df = pd.DataFrame({"Metastatic Sites":meta_sem})
# Preview DataFrame
meta_sem_df


Metastatic Sites
Drug      Timepoint                  
Capomulin 0                  0.000000
          5                  0.074833
          10                 0.125433
          15                 0.132048
          20                 0.161621
...                               ...
Zoniferol 25                 0.236621
          30                 0.248168
          35                 0.285714
          40                 0.299791
          45                 0.286400

[100 rows x 1 columns]

In [9]:
# Minor Data Munging to Re-Format the Data Frames
meta_mean_df_unstacked = meta_mean_df.unstack(level = 'Drug')['Metastatic Sites']
meta_sem_df_unstacked = meta_sem_df.unstack(level = 'Drug')['Metastatic Sites']

# Preview that Reformatting worked
meta_mean_df_unstacked.head()

Drug,Capomulin,Ceftamin,Infubinol,Ketapril,Naftisol,Placebo,Propriva,Ramicane,Stelasyn,Zoniferol
Timepoint,,,,,,,,,,
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.160000,0.380952,0.280000,0.304348,0.260870,0.375000,0.320000,0.120000,0.240000,0.166667
10,0.320000,0.600000,0.666667,0.590909,0.523810,0.833333,0.565217,0.250000,0.478261,0.500000
15,0.375000,0.789474,0.904762,0.842105,0.857143,1.250000,0.764706,0.333333,0.782609,0.809524
20,0.652174,1.111111,1.050000,1.210526,1.150000,1.526316,1.000000,0.347826,0.952381,1.294118


In [10]:
# Preview that Reformatting worked
meta_sem_df_unstacked.head()

Drug,Capomulin,Ceftamin,Infubinol,Ketapril,Naftisol,Placebo,Propriva,Ramicane,Stelasyn,Zoniferol
Timepoint,,,,,,,,,,
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.074833,0.108588,0.091652,0.098100,0.093618,0.100947,0.095219,0.066332,0.087178,0.077709
10,0.125433,0.152177,0.159364,0.142018,0.163577,0.115261,0.105690,0.090289,0.123672,0.109109
15,0.132048,0.180625,0.194015,0.191381,0.158651,0.190221,0.136377,0.115261,0.153439,0.111677
20,0.161621,0.241034,0.234801,0.236680,0.181731,0.234064,0.171499,0.119430,0.200905,0.166378


In [11]:
# Generate the Plot (with Error Bars)

#calculate the upper and lower for the error bars
drug_meta_dict = {
                 'Capomulin': {'upper':[],'lower': [], 'base':[]},
                 'Ceftamin' : {'upper':[],'lower': [], 'base':[]},
                 'Infubinol': {'upper':[],'lower': [], 'base':[]},
                 'Ketapril' : {'upper':[],'lower': [], 'base':[]},
                 'Naftisol' : {'upper':[],'lower': [], 'base':[]},
                 'Placebo'  : {'upper':[],'lower': [], 'base':[]},
                 'Propriva' : {'upper':[],'lower': [], 'base':[]},
                 'Ramicane' : {'upper':[],'lower': [], 'base':[]},
                 'Stelasyn' : {'upper':[],'lower': [], 'base':[]},
                 'Zoniferol': {'upper':[],'lower': [], 'base':[]}
                }

for key, val in drug_meta_dict.items():
    drug_meta_dict[key]['lower'].append(meta_mean_df_unstacked[key].values - meta_sem_df_unstacked[key].values)
    drug_meta_dict[key]['upper'].append(meta_mean_df_unstacked[key].values - meta_sem_df_unstacked[key].values)
    drug_meta_dict[key]['base'].append(meta_mean_df_unstacked[key].values)
    

source_meta = bp.ColumnDataSource(meta_mean_df_unstacked)

fig_2 = bp.figure(title = 'Metastatic Spread During Treatment', x_axis_label='Treatment Duration (Days)',x_range= (0,50), y_axis_label='Met. Sites', y_range=(0,4) )

i = 0
for i, c in zip(range(len(y_list)), color):
    fig_2.line('Timepoint', y_list[i], line_width=2, source=source_meta, color = c)
    source_error_meta = ColumnDataSource(drug_meta_dict[y_list[i]])
    fig_2.add_layout(Whisker(source=source_error_meta, upper ='upper', lower='lower', base = 'base', line_width = 3))

bp.output_file('Metastatic_Analysis.html', title='Metastatic Spread During Treatment') #to show in another window.
bp.save(fig_2)

try:
    bp.reset_output()
    bp.output_notebook()
    bp.show(fig_2)
except:
    bp.output_notebook()
    bp.show(fig_2)


Loading BokehJS ...

Loading BokehJS ...

![Metastatic Spread During Treatment](../Images/spread.png)

## Survival Rates

In [12]:
df = df.reset_index()
# Store the Count of Mice Grouped by Drug and Timepoint (W can pass any metric)
mouse_count = df.groupby(['Drug', 'Timepoint'])['Mouse ID'].count()
# Convert to DataFrame
mouse_count_df = pd.DataFrame({'Mouse ID': mouse_count})
# Preview DataFrame
mouse_count_df.rename(columns={'Mouse ID': 'Mouse Count'})


Mouse Count
Drug      Timepoint             
Capomulin 0                   25
          5                   25
          10                  25
          15                  24
          20                  23
...                          ...
Zoniferol 25                  16
          30                  15
          35                  14
          40                  14
          45                  14

[100 rows x 1 columns]

In [13]:
# Minor Data Munging to Re-Format the Data Frames

# Preview the Data Frame
# Minor Data Munging to Re-Format the Data Frames
mouse_count_df_unstacked = mouse_count_df.unstack(level = 'Drug')['Mouse ID']

# Preview that Reformatting worked
mouse_count_df_unstacked.head()

Drug,Capomulin,Ceftamin,Infubinol,Ketapril,Naftisol,Placebo,Propriva,Ramicane,Stelasyn,Zoniferol
Timepoint,,,,,,,,,,
0,25,25,25,25,25,25,26,25,26,25
5,25,21,25,23,23,24,25,25,25,24
10,25,20,21,22,21,24,23,24,23,22
15,24,19,21,19,21,20,17,24,23,21
20,23,18,20,19,20,19,17,23,21,17


In [14]:
# Preview the Data Frame
# Update Data Frame to turn values to percentage
mouse_percent_df_unstacked = round((((mouse_count_df_unstacked) / (mouse_count_df_unstacked.iloc[0]))*100), 2)

# Preview that Reformatting worked
mouse_percent_df_unstacked

Drug,Capomulin,Ceftamin,Infubinol,Ketapril,Naftisol,Placebo,Propriva,Ramicane,Stelasyn,Zoniferol
Timepoint,,,,,,,,,,
0,100.0,100.0,100.0,100.0,100.0,100.0,100.00,100.0,100.00,100.0
5,100.0,84.0,100.0,92.0,92.0,96.0,96.15,100.0,96.15,96.0
10,100.0,80.0,84.0,88.0,84.0,96.0,88.46,96.0,88.46,88.0
15,96.0,76.0,84.0,76.0,84.0,80.0,65.38,96.0,88.46,84.0
20,92.0,72.0,80.0,76.0,80.0,76.0,65.38,92.0,80.77,68.0
25,88.0,72.0,72.0,76.0,72.0,68.0,53.85,92.0,73.08,64.0
30,88.0,64.0,68.0,72.0,60.0,60.0,50.00,92.0,69.23,60.0
35,88.0,56.0,48.0,68.0,60.0,56.0,38.46,84.0,61.54,56.0
40,84.0,56.0,40.0,60.0,60.0,48.0,34.62,80.0,46.15,56.0


In [15]:
# Generate the Plot (Accounting for percentages)

source_surv = bp.ColumnDataSource(mouse_percent_df_unstacked)

fig_3 = bp.figure(title = 'Survival During Treatment', x_axis_label='Time (Days)',x_range= (0,50), y_axis_label='Survival Rate(%)', y_range=(40,100))
i = 0
for i, c in zip(range(len(y_list)), color):
    fig_3.line('Timepoint', y_list[i], line_width=2, source=source_surv, color = c)

bp.output_file('Survival_Rate.html', title='Survival During Treatment') #to show in another window.
bp.save(fig_3)

try:
    bp.reset_output()
    bp.output_notebook()
    bp.show(fig_3)
except:
    bp.output_notebook()
    bp.show(fig_3)


Loading BokehJS ...

Loading BokehJS ...

![Metastatic Spread During Treatment](../Images/survival.png)

## Summary Bar Graph

In [ ]:
# Calculate the percent changes for each drug

# Display the data to confirm


Drug
Capomulin   -19.475303
Ceftamin     42.516492
Infubinol    46.123472
Ketapril     57.028795
Naftisol     53.923347
Placebo      51.297960
Propriva     47.241175
Ramicane    -22.320900
Stelasyn     52.085134
Zoniferol    46.579751
dtype: float64

In [ ]:
# Store all Relevant Percent Changes into a Tuple


# Splice the data between passing and failing drugs


# Orient widths. Add labels, tick marks, etc. 


# Use functions to label the percentages of changes


# Call functions to implement the function calls


# Save the Figure


# Show the Figure
fig.show()

![Metastatic Spread During Treatment](../Images/change.png)